In [89]:
import os
import torchvision.models as models
from PIL import Image, ImageDraw, ImageFilter
import torch
import torchvision.transforms as transforms
import shutil
import numpy as np
import math
import h5py

In [2]:
images = os.listdir('E:/header-image')
print(images)

['1973320.jpg', '1973350.jpg', '1973360.jpg', '1973370.jpg', '1973390.jpg', '1973490.jpg', '1973540.jpg', '1973600.jpg', '1973630.jpg', '1973620.jpg', '1973450.jpg', '1973470.jpg', '1972670.jpg', '1973640.jpg', '1972710.jpg', '1972740.jpg', '1972820.jpg', '1972790.jpg', '1972830.jpg', '1972870.jpg', '1972880.jpg', '1972890.jpg', '1972950.jpg', '1972960.jpg', '1972970.jpg', '1972980.jpg', '1972990.jpg', '1973010.jpg', '1973110.jpg', '1973090.jpg', '1973040.jpg', '1973080.jpg', '1973130.jpg', '1973020.jpg', '1972190.jpg', '1972180.jpg', '1973150.jpg', '1972200.jpg', '1972230.jpg', '1972250.jpg', '1972260.jpg', '1972300.jpg', '1972360.jpg', '1972400.jpg', '1972410.jpg', '1972420.jpg', '1972460.jpg', '1972470.jpg', '1972550.jpg', '1972630.jpg', '1972640.jpg', '1971810.jpg', '1971800.jpg', '1971790.jpg', '1971770.jpg', '1971750.jpg', '1971880.jpg', '1971830.jpg', '1971820.jpg', '1971930.jpg', '1971980.jpg', '1971990.jpg', '1972010.jpg', '1972020.jpg', '1972050.jpg', '1972160.jpg', '1971260.

In [3]:
images.remove('images.json')

# Preprocess images

In [4]:
dir_path = 'E:/header-image'
test_images = [os.path.join(dir_path, image) for image in images[0:10]]

In [5]:
test_path = os.path.join(os.getcwd(), 'test_images')
if not os.path.exists(test_path):
    os.makedirs(test_path)

In [8]:
for image in test_images:
    new_image = os.path.join(test_path, os.path.basename(image))
    shutil.copy(image, new_image)

In [9]:
test_images = os.listdir(test_path)

In [42]:
def dominant_edge_color_padding(img):
    # Resize width to 224
    w, h = img.size
    aspect = h/w
    new_w = 224
    new_h = int(new_w * aspect)

    resized = img.resize((new_w, new_h), Image.LANCZOS)

    # Find dominant color - focus on edge regions where background is likely
    edge_width = 5  # Sample from edges
    small_img = resized.copy()
    # Create a mask that only samples from edges
    edge_mask = Image.new('L', small_img.size, 0)
    draw = ImageDraw.Draw(edge_mask)
    # Draw white rectangles around the edges
    draw.rectangle([0, 0, small_img.width-1, edge_width], fill=255)  # Top
    draw.rectangle([0, small_img.height-edge_width, small_img.width-1, small_img.height-1], fill=255)  # Bottom
    draw.rectangle([0, 0, edge_width, small_img.height-1], fill=255)  # Left
    draw.rectangle([small_img.width-edge_width, 0, small_img.width-1, small_img.height-1], fill=255)  # Right

    # Use the mask to get only edge pixels
    edge_pixels = np.array(small_img)
    edge_mask_arr = np.array(edge_mask)
    edge_pixels = edge_pixels[edge_mask_arr == 255]

    # Find most common color in edge regions
    pixels = edge_pixels.reshape(-1, 3)
    pixel_count = {}
    for pixel in pixels:
        pixel_tuple = tuple(pixel)
        if pixel_tuple in pixel_count:
            pixel_count[pixel_tuple] += 1
        else:
            pixel_count[pixel_tuple] = 1

    # Get the most common color
    dominant_color = max(pixel_count.items(), key=lambda x: x[1])[0]

    # Pad with dominant color
    padded = Image.new("RGB", (224, 224), dominant_color)
    padded.paste(resized, (0, (224-new_h)//2))
    return padded

In [45]:
def reflection_padding(img):
    # Resize width to 224
    w, h = img.size
    aspect = h/w
    new_w = 224
    new_h = int(new_w * aspect)

    resized = img.resize((new_w, new_h), Image.LANCZOS)
    resized_array = np.array(resized)

    # Calculate padding needed
    pad_top = (224 - new_h) // 2
    pad_bottom = 224 - new_h - pad_top

    # Use numpy's pad with reflection mode
    padded_array = np.pad(
        resized_array,
        ((pad_top, pad_bottom), (0, 0), (0, 0)),
        mode='reflect'
    )

    return Image.fromarray(padded_array)

In [57]:
def self_tiling_padding(img):
    # Resize width to 224 pixels
    w, h = img.size
    aspect = h/w
    new_w = 224
    new_h = int(new_w * aspect)

    # Resize the image
    resized = img.resize((new_w, new_h), Image.LANCZOS)

    # Create a new square canvas
    padded = Image.new("RGB", (224, 224))

    # Calculate how many times we need to tile vertically
    needed_height = 224
    num_tiles = math.ceil(needed_height / new_h)

    # Paste multiple copies of the image vertically
    for i in range(num_tiles):
        y_position = i * new_h
        padded.paste(resized, (0, y_position))

    # Crop to exactly 224x224
    padded = padded.crop((0, 0, 224, 224))

    return padded

In [60]:
def center_tiling_with_blur(img):
    # Resize width to 224 pixels
    w, h = img.size
    aspect = h/w
    new_w = 224
    new_h = int(new_w * aspect)

    # Resize the image
    resized = img.resize((new_w, new_h), Image.LANCZOS)

    # Create a new square canvas
    padded = Image.new("RGB", (224, 224))

    # Calculate vertical padding needed
    top_padding = (224 - new_h) // 2

    # First pass: fill the entire canvas with tiled content
    # Tile vertically both above and below the center
    for y_offset in range(0, 224, new_h):
        # Adjust to fill from top to bottom
        y_position = y_offset - (y_offset % new_h)
        padded.paste(resized, (0, y_position))

    # Second pass: paste the main content in the center (overwriting the tiled region)
    main_y_position = top_padding
    padded.paste(resized, (0, main_y_position))

    # Create a mask for the center region (where the original content is)
    mask = Image.new("L", (224, 224), 0)  # Start with black mask
    mask_draw = ImageDraw.Draw(mask)

    # Draw white rectangle for the center (original) region - fully opaque
    mask_draw.rectangle([(0, main_y_position), (224, main_y_position + new_h)], fill=255)

    # Create gradient edges for top and bottom padded regions
    fade_height = 30  # Height of the fade region

    # Top gradient (if there's padding at the top)
    if top_padding > 0:
        for y in range(fade_height):
            # Calculate alpha value (0 at the top, increasing toward the center)
            if y < top_padding:
                alpha = int(255 * y / fade_height)
                y_pos = top_padding - y
                if y_pos >= 0:
                    mask_draw.rectangle([(0, y_pos), (224, y_pos)], fill=alpha)

    # Bottom gradient (if there's padding at the bottom)
    bottom_padding_start = main_y_position + new_h
    if bottom_padding_start < 224:
        for y in range(fade_height):
            if y < (224 - bottom_padding_start):
                alpha = int(255 * (fade_height - y) / fade_height)
                y_pos = bottom_padding_start + y
                if y_pos < 224:
                    mask_draw.rectangle([(0, y_pos), (224, y_pos)], fill=alpha)

    # Apply blur to the padded regions
    # First, create a copy that will have blurred padding
    blurred = padded.copy()
    blurred = blurred.filter(ImageFilter.GaussianBlur(radius=3))

    # Final composite: use the mask to combine the original center with blurred padding
    result = Image.composite(padded, blurred, mask)

    return result

In [62]:
for image in test_images:
    image_path = os.path.join(test_path, image)
    img = Image.open(image_path).convert('RGB')
    padded = self_tiling_padding(img)
    padded_path = os.path.join(test_path, f'padded_{os.path.basename(image)}')
    padded.save(padded_path)

# Extract features

In [63]:
# Extract features from all item images
# Setup a pre-trained model (e.g., ResNet)
model = models.resnet34()
# Remove classification layer
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
feature_extractor.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [74]:
# Extract features for each item
image_features = {}

preprocess = transforms.Compose([
    transforms.Lambda(self_tiling_padding),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

for image_path in images[:10]:
    # Load and preprocess image
    image_path = os.path.join(dir_path, image_path)
    img = Image.open(image_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)

    # Extract features
    with torch.no_grad():
        features = feature_extractor(img_tensor)
        features = features.view(-1).numpy()  # Flatten to 1D

    item_id = os.path.splitext(os.path.basename(image_path))[0]
    image_features[item_id] = features

# Storing features

In [85]:
# Save features
with h5py.File('image_features.h5', 'w') as f:
    # Create a dataset with your features
    f.create_dataset('features', data=list(image_features.values()))
    # You can store item IDs or other metadata
    f.create_dataset('item_ids', data=list(image_features.keys()))

# # Load features
# with h5py.File('image_features.h5', 'r') as f:
#     loaded_features = f['features'][:]
#     loaded_item_ids = f['item_ids'][:]